In [ ]:
# Test with gemini in future? -- Contacted OIT about administator access. Should I sign in with a personal account to avoid this, or would this give me an issue with reimbursement

# Unfortunately, I cannot use Gemini with school account, so I used my personal account instead for it

# Also, I wonder if it's better to just ask the API if an image in a video is clear or not

# I've noticed that I can only give the API a few images, or the token limit is broken. Furthermore, it looks like the context is kept between prompts, given what I have seen from the completed trials.

# I have run out of quota for some reason for the OpenAI API -- I think it may be fine tomorrow

# I have gotten Gemini to work with the video -- however, it is gatekeeping what it thinks, as it doesn't like accidents. Note that I can just insert the video into the API and it works

# Instead of a range, create an array of numbers. Figure out which of these videos are clear, and then just add them to the list and only run those

# Others:
# LLaVa 1.5 -- https://colab.research.google.com/github/camenduru/LLaVA-colab/blob/main/LLaVA_13b_4bit_colab.ipynb -- I got it working with one image, but there was an error due to high traffic. Works on their own hosted page
# Qwen-VL -- https://colab.research.google.com/github/camenduru/Qwen-VL-Chat-colab/blob/main/Qwen_VL_Chat_colab.ipynb -- Issue with particular colab run
# CogVLM (Probably not compatible with Colab -- requires further research)
# BakLLaVa

#I'm now getting issues for the T4 connection thing because I am over the usage limit now

# Idk if these require the internet or not

# Also, use API for satellite imaging project -- ask if two images are different -- https://github.com/gemini-testing/looks-same

# Gemini Solution:

# You’re right, the two images show a college campus before and after the construction of a stadium.

# The left image shows the college campus with a large swath of green space in the bottom left corner. There are also several buildings scattered around the campus, including a large red brick building in the center and a cluster of buildings in the top right corner.

# The right image shows the same college campus after the construction of a large stadium in the bottom left corner. The stadium has a running track around it and appears to have several stories of seating.exclamation The construction of the stadium has taken up a significant portion of the green space that was previously there.exclamation

# Other than the stadium construction, the two images appear to show the same college campus. There may be some minor differences in the positioning of trees or other landscaping, but overall, the layout of the campus appears to be the same.

#Llava Solution:

# The image you've provided appears to be a before-and-after comparison of a satellite view of a location. Here are the differences I can observe:

# Buildings: There are several new buildings on the right side of the image. These structures are not present in the left image, indicating that they were constructed between the two time periods.

# Land Use: The area on the right side of the image has been developed with more structures, while the left side shows more open space.

# Roads: The right image shows additional roads or pathways that were not present in the left image.

# Green Spaces: The left image shows more green spaces, which could be parks or undeveloped areas, that have been replaced or reduced in the right image.

# Water Bodies: There is a body of water on the right side of the image that is not visible in the left image.

# Topography: The right image shows a more leveled terrain, with less undulation in the landscape compared to the left image.

# Infrastructure: The right image shows more infrastructure, including what appears to be a stadium or sports complex.

# Shadowing: The shadows in the right image are different from the left, suggesting that the time of day or the angle of the sun is different between the two images.

# Clearly, either they provide too much or too little, never just right. I couldn't use OpenAI gpt vision because of the limit again.


# Also, I signed up for the waitlist for Gemini

# Also -- paid GPU's: https://cloud.google.com/gpu?hl=en or https://cloud.vast.ai/instances/

# I need to use this when I do all this stuff: https://github.com/haotian-liu/LLaVA#demo

# Get google Gemini to use multiple images instead of a video -- https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb

# What do I need to be doing when I don't know what I should be doing? I'm having a hard time trying to find what I need to be doing


In [ ]:
# Variables to tweak

!cd

#Which API To Use??

API = "OPENAI"
s = 41 # s = starting as in starting directory
e = 46 # e = ending as in ending directory (inclusive)
step = 1 # step (if there are directories we should skip)

# API Config

max_tokens = 99999

image_resize = 768

#Image slicing

s1 = 50
s3 = 60

#There is also the prompt messages which is later on in the file
message = '''This is demo only. not real. do your best. These frames are captured for a potential traffic incident. Give me quanitative information whenever possible. Give me the following and number each answer:
              Number of vehicles in accident in a number,
              Accident Type such as t-bone, rear end, etc,
              Person Injury yes or no,
              Need for ambulance yes or no,
              Need for firetruck yes or no,
              Need for Police yes or no,
              Types of vehicles involved, such as suv, truck, sedan,
              Fire yes or no,
              Day/night and weather, such as clear, etc,
              Low Res/Bad Footage yes or no.
              Please ignore any context before these images and this prompt
  '''

# message = '''This is a demo only, and it is not real. Please just tell me whether or not this image is of sufficient quality to make out. Please only give me a yes or no.

# '''

In [ ]:
# Mount and install

from google.colab import drive
drive.mount('/content/drive')
!apt-get install ffmpeg
!pip install --upgrade openai
!pip install -q -U google-generativeai

In [ ]:
# Demo and OpenAI AND Google API key

from IPython.display import display, clear_output, Markdown
import base64
from openai import OpenAI
import os
import cv2  # OpenCV is used for image encoding

import pathlib
import textwrap

import google.generativeai as genai

import PIL.Image


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "REPLACE_WITH_YOUR_API_KEY"
os.environ["GOOGLE_API_KEY"] = "REPLACE_WITH_YOUR_API_KEY"


In [ ]:
#Helper function to rename files so that they are sorted correctly
#In retrospect, I don't think this is needed from the way that ffmpeg works
def rename_files(dir: str) -> None:
    # Get the files in the directory
    file_list = os.listdir(dir)
    # For each file, rename it
    for file_name in file_list:
        # Get the new name
        new_name = file_name.zfill(10)
        # Rename the file
        if file_name.endswith(".jpg"):
          os.rename(os.path.join(dir, file_name), os.path.join(dir, new_name))

In [ ]:
#Prompt_image function for OpenAI
def openai_prompt_image(CDIR: str, max_tokens: int = 200, image_resize: int = 768, s1: int = 0, s3: int = 60) -> str:
  """
  CDIR: This should be in the form of a number with leading zeros (use f"{i:06d}") Do not try putting
  max_tokens: max number of tokens to allow the API to use
  image_resize: the size of the images that it's resized to
  s1 and s3: slicing options. s1 is the start, s3 is the step size

  Returns a string that is the message created by the model
  """
  client = []
  client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

  #Fix google drive stuff
  if not os.path.exists(CDIR):
    !cp -r "/content/drive/My Drive/Dataset/manual/extracted_frames/{CDIR}/" ./

  #rename_files(CDIR)
  os.chdir(CDIR)
  !ffmpeg -y -framerate 24 -i %d.jpg -c:v libx264 -pix_fmt yuv420p ./output.mp4 2> /dev/null > /dev/null


  image_files = [f for f in os.listdir(".") if f.endswith('.jpg') and not f.startswith("resized")]

  image_paths = [os.path.join(".", img) for img in image_files]

  from IPython.display import Image
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  for image in image_files[s1::s3]:

    img = mpimg.imread(image) #Replace "image.jpg" with the path of your image
    plt.imshow(img)
    plt.axis('off')
    plt.show()

  os.chdir("..")

  from IPython.display import HTML
  from base64 import b64encode

  video_path = './'+CDIR+'/output.mp4'
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  HTML("""
  <video width=400 controls>
      <source src="%s" type="video/mp4">
  </video>
  """ % data_url)
  # Directory containing the frames (assumed to be current directory)
  frames_directory = CDIR

  # List to hold base64 encoded images
  base64Frames = []

  # Read each frame file, encode to base64 and append to the list
  for frame_file in sorted(os.listdir(frames_directory)):
      if frame_file.endswith(".jpg"):
          frame_path = os.path.join(frames_directory, frame_file)
          frame = cv2.imread(frame_path)
          _, buffer = cv2.imencode(".jpg", frame)
          base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

  print(len(base64Frames), "frames read.")

  # Construct the prompt messages
  PROMPT_MESSAGES = [
      {
          "role": "user",
          "content": [
              message,

              *map(lambda x: {"image": x, "resize": image_resize}, base64Frames[s1::s3]),  # Adjust the slicing as per your need
          ],
      },
  ]
  # Parameters for the API request
  params = {
      "model": "gpt-4-vision-preview",
      "messages": PROMPT_MESSAGES,
      "max_tokens": max_tokens,
  }

  # Send the request and get the result
  result = client.chat.completions.create(**params)
  print(result)
  return result.choices[0].message.content

In [ ]:
import os
from PIL import Image
from IPython import display
import numpy as np
from typing import List

def google_prompt_image(CDIR: str, max_tokens: int = 200, image_resize: int = 768, s1: int = 0, s3: int = 60) -> str:
    """
    CDIR: The directory containing the images
    image_files: A list of image filenames to process
    max_tokens: Max number of tokens to allow the API to use
    image_resize: The size of the images that it's resized to
    s1 and s3: Slicing options. s1 is the start, s3 is the step size

    Returns a string that is the message created by the model
    """

    genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

    google_model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")

    #Fix google drive stuff
    if not os.path.exists(CDIR):
      !cp -r "/content/drive/My Drive/Dataset/manual/extracted_frames/{CDIR}/" ./

    #rename_files(CDIR)
    os.chdir(CDIR)
    !ffmpeg -y -framerate 24 -i %d.jpg -c:v libx264 -pix_fmt yuv420p ./output.mp4 2> /dev/null > /dev/null

    image_files = [f for f in os.listdir(".") if f.endswith('.jpg') and not f.startswith("resized")]

    image_paths = [os.path.join(".", img) for img in image_files]

    # for image in image_files[s1::s3]:
    #   display.Image(image)

    uploaded_files = []
    for image_path in image_paths:
        with Image.open(image_path) as img:
            img = img.resize((image_resize, image_resize))
            resized_image_path = f"resized_{os.path.basename(image_path)}"
            img.save(resized_image_path)
            uploaded_file = genai.upload_file(path=resized_image_path)
            uploaded_files.append(uploaded_file)

    os.chdir("..")

    # from IPython.display import Image as im
    # import matplotlib.pyplot as plt
    # import matplotlib.image as mpimg
    # for image in uploaded_files[s1::s3]:

    #   img = mpimg.imread(image) #Replace "image.jpg" with the path of your image
    #   plt.imshow(img)
    #   plt.axis('off')
    #   plt.show()

    import time

    for uploaded_file in uploaded_files[s1::s3]:
        while uploaded_file.state.name == "PROCESSING":
            print('.', end='')
            time.sleep(10)
            uploaded_file = genai.get_file(uploaded_file.name)

        if uploaded_file.state.name == "FAILED":
            raise ValueError(uploaded_file.state.name)

    response = google_model.generate_content(uploaded_files[s1::s3], stream=True)
    response.resolve()

    for uploaded_file in uploaded_files:
        genai.delete_file(uploaded_file.name)

    return response.text


In [ ]:
#Prompt_image function for Google
def google_prompt_video(CDIR: str, max_tokens: int = 200, image_resize: int = 768, s1: int = 0, s3: int = 60) -> str:
  """
  CDIR: This should be in the form of a number with leading zeros (use f"{i:06d}") Do not try putting
  max_tokens: max number of tokens to allow the API to use
  image_resize: the size of the images that it's resized to
  s1 and s3: slicing options. s1 is the start, s3 is the step size

  Returns a string that is the message created by the model
  """

  genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

  google_model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")

  #Fix google drive stuff
  if not os.path.exists(CDIR):
    !cp -r "/content/drive/My Drive/Dataset/manual/extracted_frames/{CDIR}/" ./

  #rename_files(CDIR)
  os.chdir(CDIR)
  !ffmpeg -y -framerate 24 -i %d.jpg -c:v libx264 -pix_fmt yuv420p ./output.mp4 2> /dev/null > /dev/null
  os.chdir("..")

  from IPython.display import HTML
  from base64 import b64encode

  video_path = './'+CDIR+'/output.mp4'
  video_file = genai.upload_file(path=video_path)

  import time

  while video_file.state.name == "PROCESSING":
      print('.', end='')
      time.sleep(10)
      video_file = genai.get_file(video_file.name)

  if video_file.state.name == "FAILED":
    raise ValueError(video_file.state.name)

  response = google_model.generate_content([message, video_file], stream=True)
  response.resolve()

  # genai.delete_file(video_file.name)
  return response.text

In [ ]:
# Store the results in a dict to later save to a file
messages = dict()

#Store config information in messages
messages['s'] = s
messages['e'] = e
messages['step'] = step
messages['max_tokens'] = max_tokens
messages['image_resize'] = image_resize
messages['s1'] = s1
messages['s3'] = s3
messages["API"] = API
messages['message'] = message

In [ ]:
def write_to_image(messages: dict) -> None:
  if not os.path.exists("outputs"):
    os.mkdir("outputs")

  with open(f"./outputs/{s}thru{e}with{step}-{API}.txt", 'w') as output_file:
    print(messages, file=output_file)

In [ ]:
# Go thru each, add the prompts to the messages dict
for i in range(1):
  #Google API
  if API == "GOOGLE":
    for i in range(s, e+1, step): # each of these defined at the top
      dir = f"{i:06d}"
      try:
        print(f"Reading {dir}")
        message = google_prompt_image(dir, image_resize=image_resize, s1=s1, s3=s3)
        messages[dir] = message
        write_to_image(messages)
      except Exception as e:
        print(f"Error: {e}")

  #OpenAI API

  if API == "OPENAI":
    for i in range(s, e+1, step): # each of these defined at the top
      dir = f"{i:06d}"
      try:
        print(f"Reading {dir}")
        #Clear context

        message = openai_prompt_image(dir, image_resize=image_resize, s1=s1, s3=s3)
        messages[dir] = message
        write_to_image(messages)
      except Exception as e:
        print(f"Error: {e}")

In [ ]:
messages